# Electricity Price Forecasting on the German day-ahead market

This notebook is the main interface to the associated EPF library. Various parameters can be adjusted within the library via the Config file. A more detailed description of the individual parameters can be found within the configuration classes. In the “exploratory_analysis” notebook, the raw data sets are analyzed exploratively and the results are displayed visually. Based on these results, features from the data sets were specifically selected for further use in the deep learning pipeline.

The forecasting pipeline is built to automatically perform data preprocessing including data cleaning, outlier removal and seasonal decomposition. Within the configuration feature engineering can be toggled on and of for each feature. Forecasting is done with two different models, that can be retrained and saved anytime. The models available are an LSTM and GRU. They all perform multi-step ahead, single shot forecasts. This means by default each forecast contains 24 time steps into the future, that are forecasted together in one computation.

In [1]:
from epf.pipeline import EpfPipeline
import pickle as pkl

pipeline = EpfPipeline()

2025-05-30 15:08:33.961 | INFO     | epf.config:<module>:19 - PROJ_ROOT path is: C:\Users\valen\PycharmProjects\epf
2025-05-30 15:08:33.961 | INFO     | epf.config:create_dir:15 - DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data
2025-05-30 15:08:33.976 | INFO     | epf.config:create_dir:15 - RAW_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/raw
2025-05-30 15:08:33.976 | INFO     | epf.config:create_dir:15 - INTERIM_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/interim
2025-05-30 15:08:33.976 | INFO     | epf.config:create_dir:15 - PROCESSED_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/processed
2025-05-30 15:08:33.976 | INFO     | epf.config:create_dir:15 - TRAIN_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/processed/train_data
2025-05-30 15:08:33.976 | INFO     | epf.config:create_dir:15 - MODELS_DIR path is: C:/Users/valen/PycharmProjects/epf/models
2025-05-30 15:08:33.976 | INFO     | epf.config:create_dir:15 - PREDICTIONS_D

Pipeline initialized with:
FeatureConfig: 
Selected features: 
['DE-LU Prices', 'DE Solar Generation', 'DE Wind Generation Offshore', 'DE Wind Generation Onshore', 'DE Load', 'Month', 'Day of Week', 'Holiday']
WINDOW_LENGTH: 24
N_SIGMA: 3
METHOD: nearest

ModelConfig: 
TRAIN_SPLIT (upper bound): 2023-09-30 22:00:00+00:00
VALIDATION_SPLIT (upper bound): 2023-12-31 22:00:00+00:00
MAX_EPOCHS: 100
OUT_STEPS: 24
SEASONALITY_PERIOD: 24
INPUT_WIDTH_FACTOR: 1.25
MODEL_BUILDER: LSTM
USE_HIDDEN_LAYERS: False
NUM_FEATURES: 8
UNIT_MIN_VALUE: 32
UNIT_MAX_VALUE: 128
UNIT_STEP: 32
LR_MIN_VALUE: 0.0001
LR_MAX_VALUE: 0.1
LR_STEP: 0.001
DROPOUT_RATE_MIN_VALUE: 0.2
DROPOUT_RATE_MAX_VALUE: 0.7
DROPOUT_RATE_STEP: 0.05
NUM_LAYERS_MIN: 1
NUM_LAYERS_MAX: 3
NUM_LAYERS_STEP: 1
MAX_TRIALS: 50
LABEL_COL: de_prices_hat_rm_seasonal




In [2]:
model_name = 'shifted_lstm_vre_sl'

pipeline.train(model_name, overwrite=True)
pipeline.evaluate(model_name)

Trial 50 Complete [00h 00m 14s]
mean_absolute_error: 0.046248964965343475

Best mean_absolute_error So Far: 0.04271925985813141
Total elapsed time: 00h 11m 55s


C:\Users\valen\PycharmProjects\epf\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-05-30 15:23:29.040 | INFO     | epf.pipeline:_tune_hyperparameters:325 - Hyperparameter tuning took 00h:11m:56s.
2025-05-30 15:23:29.040 | SUCCESS  | epf.pipeline:train:398 - Successfully tuned hyperparameters for shifted_lstm_vre_sl
2025-05-30 15:23:29.040 | INFO     | epf.pipeline:train:405 - Training shifted_lstm_vre_sl on the following Features:['DE-LU Prices', 'DE Solar Generation', 'DE Wind Generation Offshore', 'DE Wind Generation Onshore', 'DE Load', 'Month', 'Day of Week', 'Holiday'].


Epoch 1/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0449 - mean_absolute_error: 0.0449 - root_mean_squared_error: 0.0594 - val_loss: 0.0614 - val_mean_absolute_error: 0.0614 - val_root_mean_squared_error: 0.0861
Epoch 2/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0419 - mean_absolute_error: 0.0419 - root_mean_squared_error: 0.0561 - val_loss: 0.0609 - val_mean_absolute_error: 0.0609 - val_root_mean_squared_error: 0.0835
Epoch 3/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0419 - mean_absolute_error: 0.0419 - root_mean_squared_error: 0.0562 - val_loss: 0.0600 - val_mean_absolute_error: 0.0600 - val_root_mean_squared_error: 0.0823
Epoch 4/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0420 - mean_absolute_error: 0.0420 - root_mean_squared_error: 0.0561 - val_loss: 0.0603 - val_mean_absolute_error: 0.0603 - val_root_mean_squared_error: 0.0846
Epoch 5/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0406 - mean_absolute_error: 0.0406 - root_mea

2025-05-30 15:23:40.784 | SUCCESS  | epf.pipeline:train:412 - Successfully trained shifted_lstm_vre_sl. Now saving...
2025-05-30 15:23:40.784 | INFO     | epf.pipeline:train:417 - Training took 00h:15m:02s.
2025-05-30 15:23:40.816 | SUCCESS  | epf.pipeline:train:442 - Successfully saved shifted_lstm_vre_sl to C:/Users/valen/PycharmProjects/epf/models/shifted_lstm_vre_sl.pkl
2025-05-30 15:23:40.816 | INFO     | epf.pipeline:evaluate:454 - Loading trained model from C:\Users\valen\PycharmProjects\epf\models\shifted_lstm_vre_sl.pkl.


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0621 - mean_absolute_error: 0.0621 - root_mean_squared_error: 0.0853


2025-05-30 15:23:42.051 | INFO     | epf.pipeline:evaluate:487 - Evaluation took 00h:00m:01s.
